# Loss Function
* A loss function takes the **(output,target)** pair of inputs, and computes a value that estimates how far away the output is from the target.
* This package : https://pytorch.org/docs/stable/nn.html
* There are several different loss functions under the nn package . **A simple loss is: nn.MSELoss which computes the mean-squared error between the input and the target.**




In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [10]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0312,  0.0350,  0.0042, -0.0705,  0.0573,  0.0291,  0.0285, -0.0828,
          0.0359, -0.0251]], grad_fn=<AddmmBackward>)


In [12]:
output = net(input)
target = torch.randn(10) # a dummy target, for example
target = target.view(1, -1) # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.7861, grad_fn=<MseLossBackward>)


In [21]:
help(loss.grad_fn)

Help on MseLossBackward object:

class MseLossBackward(object)
 |  Methods defined here:
 |  
 |  __call__(self, /, *args, **kwargs)
 |      Call self as a function.
 |  
 |  name(...)
 |  
 |  register_hook(...)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  metadata
 |  
 |  next_functions
 |  
 |  requires_grad



In [24]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])# Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) # Relu

## Back prop
* To backpropagate the error all we have to do is to **loss.backward()**. You need to clear the existing gradients though, else gradients will be accumulated to existing gradients.
* backpropagate (誤差逆伝播法)
* Now we shall call **loss.backward()**, and have a look at conv1’s bias gradients before and after the backward.

In [25]:
net.zero_grad() # zeroes the gradient buffers of all parameters
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)
loss.backward()
print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
None
conv1.bias.grad after backward
tensor([-0.0174,  0.0065,  0.0150, -0.0067, -0.0036, -0.0139])


* Now, we have seen how to use loss functions.
* The neural network package contains various modules and loss functions that form the building blocks of deep neural networks. A full list with documentation is here.

# Update the weights
* The simplest update rule used in practice is the Stochastic Gradient Descent (SGD):
    * **weight = weight - learning_rate * gradient**

In [26]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

* However, as you use neural networks, you want to use various different update rules such as SGD, Nesterov-SGD, Adam, RMSProp, etc. 
* To enable this, we built a small package: **torch.optim** that implements all these methods. Using it is very simple:

In [27]:
import torch.optim as optim
# create your optimizer
optimizer = optim.SGD(net.parameters(), lr = 0.01)
# in your training loop:
optimizer.zero_grad() # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step() # Does the update

* Observe how gradient buffers had to be manually set to zero using optimizer.zero_grad(). 
* This is because **gradients are accumulated as explained in the Backprop section.**